# ARTisan

Useful resources
- [A Neural Algorithm of Artistic Style (2015)](https://arxiv.org/pdf/1508.06576.pdf)
- [Neural Style Transfer with tf.keras](https://aihub.cloud.google.com/u/0/p/products%2F7f7495dd-6f66-4f8a-8c30-15f211ad6957)
- [Salvador AI (Old course project)](https://dellacqua.se/education/courses/tnm095/archive/2017/p10.pdf)
- [Style transfer keras example](https://keras.io/examples/generative/neural_style_transfer/)
- [Demystifying Neural Style Transfer](https://arxiv.org/pdf/1701.01036.pdf)

## Test if GPU is available


In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())

## Setup and getting the images

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import vgg19
from tensorflow import keras
from IPython.display import Image, display

print('Tensorflow version:', tf.__version__)

content_img_path = keras.utils.get_file(
    "trams.jpg", "https://www.svtstatic.se/image/wide/992/15663340/1575374592?format=auto")

# content_img_path = keras.utils.get_file(
#     "rabbel.jpg", "https://static.wikia.nocookie.net/vintergatan/images/f/fd/Rabbel.jpg/revision/latest/scale-to-width-down/1000?cb=20201127071427&path-prefix=sv")
    
style_img_path = keras.utils.get_file(
    "starry_night.jpg", "https://i.imgur.com/9ooB60I.jpg")

display(Image(content_img_path))
#display(Image(style_img_path))

content_width, content_height = keras.preprocessing.image.load_img(content_img_path).size
n_rows = 400
n_cols = int(content_width * n_rows / content_height)

## Utilities for preprocessing/deprocessing
The VGG19 network expects the data to be in a certain format. More specifically, it expects
- the input to be in BGR rather than RGB
- the input to be zero-centered with respect to the mean of the ImageNet dataset

In [ ]:
def preprocess_img(img_path):
  img = keras.preprocessing.image.load_img(
      img_path, target_size=(n_rows, n_cols))
  img = keras.preprocessing.image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return tf.convert_to_tensor(img)

def deprocess_img(x):
  x = x.reshape((n_rows, n_cols, 3))
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1] # BGR -> RGB
  x = np.clip(x, 0, 255).astype("uint8")
  return x

## Utilities for loss calculations


In [ ]:
def content_loss(content, combination):
  return tf.reduce_sum(tf.square(combination - content))

# TODO

## Prepare model etc.

In [ ]:
model = vgg19.VGG19(weights="imagenet", include_top=False) # Using pretrained weights from imagenet
outputs = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs)

In [ ]:
#content_weight = 2.5e-8
content_weight = 2.5e-5
# content_layer_name = "block5_conv2"
content_layer_name = "block5_conv2"


def compute_loss(combination_img, content_img, style_img):
  # Concatenate tensors along dimension 0
  input_tensor = tf.concat([content_img, style_img, combination_img], axis=0)
  #
  features = feature_extractor(input_tensor)

  loss = tf.zeros(shape=()) # WHY shape=()

  # TODO Style loss, total variation loss also I think

  # Content loss
  layer_features = features[content_layer_name]
  content_features = layer_features[0, :, :, :]
  combination_features = layer_features[2, :, :, :]
  loss = loss + content_weight * content_loss(content_features, combination_features)

  return loss

## Compute gradient and loss util
Created as a tensorflow function to improve performance, as described [here](https://www.tensorflow.org/guide/function)

In [ ]:
@tf.function
def compute_loss_and_gradients(combination_img, content_img, style_img):
  with tf.GradientTape() as tape:
    loss = compute_loss(combination_img, content_img, style_img)
  gradients = tape.gradient(loss, combination_img)
  return loss, gradients

## The optimization loop

In [ ]:
#num_iterations = 4000
#save_interval = 100
num_iterations = 10000
save_interval = 100

learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 100.0,
    decay_steps = 100,
    decay_rate = 1.00 # 0.96
)
optimizer = keras.optimizers.SGD(learning_rate_schedule)

content_img = preprocess_img(content_img_path)
style_img = preprocess_img(style_img_path)
#combination_img = tf.Variable(preprocess_img(content_img_path))
combination_img = tf.Variable(tf.random.uniform(
    content_img.shape, minval = -128, maxval = 128 # No idea what im doing
))
keras.preprocessing.image.save_img("start.png", deprocess_img(combination_img.numpy()))
display(Image("start.png"))


for i in range(1, num_iterations + 1):
  loss, gradients = compute_loss_and_gradients(combination_img, content_img, style_img)

  optimizer.apply_gradients([(gradients, combination_img)])

  if i % save_interval == 0 or i == num_iterations:
    print(f"Iteration {i}, loss = {loss}")
    img = deprocess_img(combination_img.numpy())
    file_name = f"result-at-{i}-iterations.png"
    keras.preprocessing.image.save_img(file_name, img)
    display(Image(f"result-at-{i}-iterations.png"))